# MIT-GSL Uruguay 

## January 2020

-----

# Week - 2 | Lesson - 03 
# NLP: Product embeddings

1. After introducing the concept of embeddings through NLP, we extend the notion of embeddings to other settings
2. Note that embeddings are dense continuous representations for discrete, sparse tokens - this makes embeddings widely applicable
3. We will use the concept of embeddings to understand the world of e-commerce better 

---

# Word2Vec and its Applications to Market-Basket Data

# Instacart Grocery Dataset

### Source: https://www.instacart.com/datasets/grocery-shopping-2017

1. Instacart is an online grocery delivery service
2. They have made available 3M grocery orders for over 200K users
3. They provide between 4 to 100 orders for each user and each order contains the sequence of products purchased
4. We also have a brief description of the products

### Overview:
1. We will use this data to build an understanding of word embeddings and investigate their application to downstream tasks
2. For this purpose, we will consider each purchase basket to be a sentence with an unordered sequence of words

---

# 0. Import Modules

In [1]:
# ==============================================
# 0. Module imports
# ==============================================

import pandas as pd
pd.options.display.max_colwidth = 100
import numpy as np
from itertools import product
import csv

# w2v
import gensim

# text processing
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import ToktokTokenizer
import string
import re # regular expressions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity

# T-Sne
#import umap
#from openTSNE import TSNE, TSNEEmbedding, affinity, initialization
#from openTSNE import initialization
#from openTSNE.callbacks import ErrorLogger
from sklearn.manifold import TSNE


# utils
from sklearn import utils


# parallel processing
import multiprocessing
from joblib import delayed, Parallel

# time code
import time

# 2-d visualiztion
%matplotlib inline
from ggplot import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import os
# os.chdir("/pool001/madhavk/gsl-uruguay/W-02-NLP/")

In [2]:
# =========================================================
# set directories
# =========================================================

import os
EC2 = True  # If using EC2 (for data directory/paths)

# Select path based off of local or remote
if not EC2:
    wd = "/pool001/madhavk/gsl-uruguay/W-02-NLP/"
else:
    wd = "/home/ubuntu/machine_learning_aws/"
os.chdir(wd)

EC2 = True  # If using EC2 (for data directory/paths)
if not EC2:
    # raw data
    raw_data_dir = "nlp-data/in-grocery/instacart_2017_05_01/"
    # processed data
    process_dir = "nlp-data/in-grocery/prepared-data/"  
else:
    # raw data
    raw_data_dir = "data/in-grocery/instacart_2017_05_01"
    # processed data
    process_dir = "data/in-grocery/prepared-data/"

In [3]:
# =========================================================
# processed files
# =========================================================

os.listdir(process_dir)

['orders-split-v1.csv',
 'all-orders-wide-v1.csv',
 'all-orders-long-v1.csv',
 'products-merged-v1.csv']

In [4]:
# =========================================================
# global parameters
# =========================================================

# show entire value of cell in pandas
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

# number of cpus
cpus = multiprocessing.cpu_count()
f"Number of CPUs: {cpus}"

'Number of CPUs: 4'

------

# 1. Import grocery data

## 1.1. Order level data

In [5]:
# =========================================================
# order-level data
# =========================================================

orders_wide = pd.read_csv(process_dir + "all-orders-wide-v1.csv")
# This data set has one row per order with the products ordered in the product_id column. 
# Products are separated by space.
print(orders_wide.shape)

(3152555, 4)


In [6]:
display(orders_wide.head(10))

,order_id,product_id,num_products,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,prior
4,7,34050 46802,2,prior
5,9,21405 47890 11182 2014 29193 34203 14992 31506 23288 44533 18362 432 3990 14183,14,prior
6,10,24852 4796 31717 47766 4605 1529 21137 22122 34134 27156 14992 49235 26842 3464 25720,15,prior
7,11,30162 5994 1313 31506,4,prior
8,12,30597 15221 43772 37886 37215 34335 26910 38888 38050 29471,10,prior
9,13,17330 27407 35419 196 44635 26878 25783 41290 33198 23020 36086 3800 25952,13,prior


In [7]:
# =========================================================
# orders meta-data
# =========================================================

orders_meta = pd.read_csv(process_dir + "orders-split-v1.csv")
# This dataset includes the meta data for each order, i.e., the user who ordered it, order day of the week, order time
print(orders_meta.shape)

(3346083, 7)


In [8]:
display(orders_meta.head(10))

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval
0,2539329,1,1,2,8,NaN,prior
1,2398795,1,2,3,7,15.0,prior
2,473747,1,3,3,12,21.0,prior
3,2254736,1,4,4,7,29.0,prior
4,431534,1,5,4,15,28.0,prior
5,3367565,1,6,2,7,19.0,prior
6,550135,1,7,1,9,20.0,prior
7,3108588,1,8,1,14,14.0,prior
8,2295261,1,9,1,16,0.0,prior
9,2550362,1,10,4,8,30.0,prior


----

## 1.2. Merge train-val-test split

In [9]:
# =========================================================
# merge orders-wide and orders-meta
# =========================================================

orders_wide.drop(labels = "eval", axis = 1, inplace = True)
orders_wide = pd.merge(orders_wide, # data - 1
                       orders_meta[["order_id", "user_id", "eval"]], # data - 2
                       on = "order_id", # merge key
                       how = "left") # left join
print(orders_wide.shape)

(3152555, 5)


In [10]:
display(orders_wide.head(10))

,order_id,product_id,num_products,user_id,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,202279,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,205970,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,178520,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,156122,prior
4,7,34050 46802,2,142903,prior
5,9,21405 47890 11182 2014 29193 34203 14992 31506 23288 44533 18362 432 3990 14183,14,139016,prior
6,10,24852 4796 31717 47766 4605 1529 21137 22122 34134 27156 14992 49235 26842 3464 25720,15,135442,prior
7,11,30162 5994 1313 31506,4,143742,prior
8,12,30597 15221 43772 37886 37215 34335 26910 38888 38050 29471,10,152610,prior
9,13,17330 27407 35419 196 44635 26878 25783 41290 33198 23020 36086 3800 25952,13,45082,prior


In [13]:
# =========================================================
# eval-set distribution
# =========================================================

orders_wide["eval"].value_counts()

prior    2959079
train    116333 
val      38701  
test     38442  
Name: eval, dtype: int64

In [12]:
orders_wide["aisle"].value_counts()

KeyError: 'aisle'

In [84]:
# clear some space
del orders_meta

----

## 1.3. Import product info data

In [14]:
products = pd.read_csv(process_dir + "products-merged-v1.csv")
print(products.shape)

(49688, 6)


In [15]:
display(products.head(10))

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry
5,6,Dry Nose Oil,11,11,cold flu allergy,personal care
6,7,Pure Coconut Water With Orange,98,7,juice nectars,beverages
7,8,Cut Russet Potatoes Steam N' Mash,116,1,frozen produce,frozen
8,9,Light Strawberry Blueberry Yogurt,120,16,yogurt,dairy eggs
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7,water seltzer sparkling water,beverages


In [16]:
#===============================================
# top departments
#===============================================
products["aisle"].value_counts()

missing                          1258
candy chocolate                  1246
ice cream ice                    1091
vitamins supplements             1038
yogurt                           1026
chips pretzels                   989 
tea                              894 
packaged cheese                  891 
frozen meals                     880 
cookies cakes                    874 
energy granola bars              832 
hair care                        816 
spices seasonings                797 
juice nectars                    792 
crackers                         747 
soup broth bouillon              737 
baby food formula                718 
coffee                           680 
refrigerated                     675 
cleaning products                655 
baking ingredients               623 
packaged vegetables fruits       615 
asian foods                      605 
nuts seeds dried fruit           582 
fresh vegetables                 569 
oral hygiene                     565 
salad dressi

-----

### In-class exercise

In [88]:
#===============================================
# top aisles
#===============================================
# can you figure out which aisles host the most number of products?

---

# 2. Data exploration

In [17]:
# long-form of orders
orders_long = pd.read_csv(process_dir + "all-orders-long-v1.csv")
print(orders_long.shape)

(32019330, 5)


In [19]:
# top-20 observations in the data frame
display(orders_long.head(20))

,order_id,product_id,add_to_cart_order,reordered,eval
0,2,33120,1,1,prior
1,2,28985,2,1,prior
2,2,9327,3,0,prior
3,2,45918,4,1,prior
4,2,30035,5,0,prior
5,2,17794,6,1,prior
6,2,40141,7,1,prior
7,2,1819,8,1,prior
8,3,33754,1,1,prior
9,3,24838,2,1,prior


## 2.1. Most frequently purchased products

In [24]:
# most frequently bought products
most_freq_purchased = pd.DataFrame(orders_long["product_id"].value_counts()) # count the number times each product-id appears in data frame
most_freq_purchased.reset_index(drop = False, inplace = True) # complying with pandas indexing 
most_freq_purchased.columns = ["product_id", "freq"] # assign column names
display(most_freq_purchased.head(10))

,product_id,freq
0,24852,491291
1,13176,394930
2,21137,275577
3,21903,251705
4,47209,220877
5,47766,184224
6,47626,160792
7,16797,149445
8,26209,146660
9,27845,142813


In [25]:
# merge with product info from the meta dataset
most_freq_purchased = pd.merge(most_freq_purchased, 
                               products, 
                               on = "product_id", 
                               how = "left")
display(most_freq_purchased.head(20))

,product_id,freq,product_name,aisle_id,department_id,aisle,department
0,24852,491291,Banana,24,4,fresh fruits,produce
1,13176,394930,Bag of Organic Bananas,24,4,fresh fruits,produce
2,21137,275577,Organic Strawberries,24,4,fresh fruits,produce
3,21903,251705,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
4,47209,220877,Organic Hass Avocado,24,4,fresh fruits,produce
5,47766,184224,Organic Avocado,24,4,fresh fruits,produce
6,47626,160792,Large Lemon,24,4,fresh fruits,produce
7,16797,149445,Strawberries,24,4,fresh fruits,produce
8,26209,146660,Limes,24,4,fresh fruits,produce
9,27845,142813,Organic Whole Milk,84,16,milk,dairy eggs


----

### In-class exercise

In [26]:
display(most_freq_purchased.tail(20))

,product_id,freq,product_name,aisle_id,department_id,aisle,department
14320,15612,206,Unwrapped Bites,45,19,candy chocolate,snacks
14321,20434,206,Sardines in Olive Oil,95,15,canned meat seafood,canned goods
14322,22315,206,Slow Churned Chocolate Light Ice Cream,37,1,ice cream ice,frozen
14323,39328,206,Organic Mexican Chocolate Ice Cream,37,1,ice cream ice,frozen
14324,12549,206,"Salad Dressing, Bleu Cheese, Vegan",89,13,salad dressing toppings,pantry
14325,35355,205,94% Fat Free Microwave Butter Popcorn,23,19,popcorn jerky,snacks
14326,5634,205,Organic Pasture Raised Local Eggs,86,16,eggs,dairy eggs
14327,18975,205,Uncured Cherry Smoked Bacon,106,12,hot dogs bacon sausage,meat seafood
14328,20814,205,Sensitive with Iron Infant Formula,92,18,baby food formula,babies
14329,46815,205,Ultimate Omega 1280mg omega-3,47,11,vitamins supplements,personal care


In [28]:
# most popular aisle
aisle = most_freq_purchased.groupby("aisle").agg({"freq": "sum"})
aisle = aisle.reset_index(drop = False, inplace = False).sort_values("freq", ascending = False)
display(aisle)

,aisle,freq
50,fresh fruits,3782826
53,fresh vegetables,3553596
98,packaged vegetables fruits,1823815
133,yogurt,1475395
93,packaged cheese,990483
83,milk,918611
131,water seltzer sparkling water,868265
25,chips pretzels,717827
119,soy lactosefree,655592
11,bread,588261


----

### In-class exercise

In [30]:
aisle = most_freq_purchased.groupby("aisle").agg({"freq": "sum"})
aisle = aisle.reset_index(drop = False, inplace = False).sort_values("freq", ascending = False)
display(aisle)

,aisle,freq
50,fresh fruits,3782826
53,fresh vegetables,3553596
98,packaged vegetables fruits,1823815
133,yogurt,1475395
93,packaged cheese,990483
83,milk,918611
131,water seltzer sparkling water,868265
25,chips pretzels,717827
119,soy lactosefree,655592
11,bread,588261


In [33]:
department = most_freq_purchased.groupby("department").agg({"freq": "sum"})
department = department.reset_index(drop = False, inplace = False).sort_values("freq", ascending = False)
display(department)

,department,freq
19,produce,9840710
7,dairy eggs,5522579
20,snacks,2767610
3,beverages,2641843
10,frozen,2178717
16,pantry,1763171
2,bakery,1168021
8,deli,1046768
6,canned goods,1032645
9,dry goods pasta,833181


-----

## 2.2. Co-purchased products

In [34]:
# ===========================================
# Generate co-purchase matrix
# ===========================================

def CoPurchaseMatrix(orders_wide, product_info = True):
    '''
    orders_wide is order-level data with one row per order
    '''
    count_vec = CountVectorizer(ngram_range = (1,1), binary = True, 
                            token_pattern = "\\b\\w+\\b") # sku counts
    pur_mat = count_vec.fit_transform(orders_wide["product_id"])
    co_pur_mat = (pur_mat.T * pur_mat) #copurchase matrix
    co_pur_mat.setdiag(0) # set diagonal to 0
    co_pur_mat_df = pd.DataFrame(co_pur_mat.todense()) # convert to data frame
    co_pur_mat_df.index = count_vec.vocabulary_ # row names from sku-ids
    co_pur_mat_df.columns = count_vec.vocabulary_ # column names from sku-ids
    co_pur_mat_df = co_pur_mat_df.where(np.triu(np.ones(co_pur_mat_df.shape)).astype(np.bool)) # consider the uppre tri
    co_pur_mat_df = co_pur_mat_df.stack().reset_index() # melt to sku-1 and sku-2 per row
    co_pur_mat_df.columns = ["product_id_1", "product_id_2", "copur"]
    co_pur_mat_df = co_pur_mat_df.loc[co_pur_mat_df["copur"] > 0, :] # subset for copur > 0
    co_pur_mat_df_top = co_pur_mat_df.sort_values(["product_id_1", "copur"], ascending = False)
    co_pur_mat_df_top = co_pur_mat_df_top.drop_duplicates(["product_id_1"], keep = "first")
    co_pur_mat_df_top["product_id_1"] = co_pur_mat_df_top["product_id_1"].astype(int) # fix data types
    co_pur_mat_df_top["product_id_2"] = co_pur_mat_df_top["product_id_2"].astype(int) # fix data types
    co_pur_mat_df_top = co_pur_mat_df_top.sort_values(["copur"], ascending = False).reset_index(drop = True)
    if product_info:
        co_pur_mat_df_top = pd.merge(co_pur_mat_df_top, products, how = "left", 
                                     left_on = "product_id_1", right_on = "product_id")
        co_pur_mat_df_top.drop("product_id", axis = 1, inplace = True)
        co_pur_mat_df_top = pd.merge(co_pur_mat_df_top, products, how = "left",
                                     left_on = "product_id_2", right_on = "product_id", 
                                     suffixes = ["_1", "_2"])
        co_pur_mat_df_top.drop("product_id", axis = 1, inplace = True)
        col_order = ['product_id_1', 'product_id_2', 'copur', 'product_name_1', 'product_name_2',
                             'aisle_1', 'aisle_2', 'department_1', 'department_2', 
                             'aisle_id_1', 'aisle_id_2', 'department_id_1', 'department_id_2']
        co_pur_mat_df_top = co_pur_mat_df_top[col_order]
    return(co_pur_mat_df_top)

In [35]:
# calculate copurchases
copur = CoPurchaseMatrix(orders_wide = orders_wide, product_info = True)
print(copur.shape)

(14332, 13)


In [36]:
display(copur[["product_name_1", "product_name_2", "copur", "aisle_1", "aisle_2"]].head(20))

,product_name_1,product_name_2,copur,aisle_1,aisle_2
0,Organic Lacinato (Dinosaur) Kale,Soft Eating Strawberry Flavored Licorice,64761.0,fresh vegetables,candy chocolate
1,"Mighty 4 Sweet Potato, Blueberry, Millet & Greek Yogurt Tots Snack",Nut Delight Fruit & Nut Bar,58330.0,baby food formula,energy granola bars
2,Nut Delight Fruit & Nut Bar,Carrot Bunch,55611.0,energy granola bars,fresh vegetables
3,Natural Finely Shredded Triple Cheddar Cheese,Nut Delight Fruit & Nut Bar,53395.0,packaged cheese,energy granola bars
4,Italian Style Meatballs & Mozzarella Sandwiches,Nut Delight Fruit & Nut Bar,43180.0,frozen meals,energy granola bars
5,Organic Granny Smith Apples,Garlic Spice Blend Paste,28998.0,fresh fruits,packaged vegetables fruits
6,Pecan Pie Fruit & Nut Food Bar,Soft Eating Strawberry Flavored Licorice,26812.0,energy granola bars,candy chocolate
7,Garlic Spice Blend Paste,Carrot Bunch,25766.0,packaged vegetables fruits,fresh vegetables
8,"Almond Coconut Bar, Organic",2nd Foods Organic Pear and Spinach Baby Food,22923.0,energy granola bars,baby food formula
9,Soft Eating Strawberry Flavored Licorice,Spicy Minis Guacamole,22027.0,candy chocolate,fresh dips tapenades


----

# 3. Split train-val-test datasets


In [37]:
#===============================================
# split train-val-test
#===============================================

train = orders_wide.loc[orders_wide["eval"].isin(["prior", "train"]), :]
val = orders_wide.loc[orders_wide["eval"] == "val", :]
test = orders_wide.loc[orders_wide["eval"] == "test", :]
print("train size:", train.shape)
print("val size:", val.shape)
print("test size:", test.shape)

train size: (3075412, 5)
val size: (38701, 5)
test size: (38442, 5)


In [101]:
# clear more space
# del orders_wide, orders_long

In [38]:
display(train.head())

,order_id,product_id,num_products,user_id,eval
0,2,33120 28985 9327 45918 30035 17794 40141 1819,8,202279,prior
1,3,33754 24838 17704 21903 17668 46667 17461 32665,8,205970,prior
2,4,46842 39758 27761 10054 21351 22598 34862 40285 17616 25146 32645 41276,12,178520,prior
3,5,13176 47329 27966 23909 48370 13245 27360 6348 40878 6184 48002 20914 37011 12962 45698 41176 48366 47209 46522 38693 48825 8479,22,156122,prior
4,7,34050 46802,2,142903,prior


## 3.1. Random sample for faster processing

In [39]:
#===============================================
# randomly sample training data
#===============================================

sample_size = 1000000
train = train.sample(n = sample_size)
train = train.reset_index(drop = True)
print(train.shape)

(1000000, 5)


In [40]:
display(train.head(10))

,order_id,product_id,num_products,user_id,eval
0,2713434,36127 260 38739 9839 31717 5212 16797 26209 5652,9,39434,prior
1,2564998,34126 34065 405 43772 43129 27966 24964 33731 49520 42736,10,123901,prior
2,390578,7677 27086 46175 4421 12276 9339 19678 42701 4605 24964,10,52624,prior
3,3171317,48364 37158 34358,3,102586,prior
4,1649445,14161 47357 20345 17341 11281 38849 25544 27676 14144 20520 36227 37722 1543,13,11004,prior
5,3409448,31506 46667 21847 20588 20638 49424 23516 48679,8,52968,prior
6,3032813,26282 10804 21344 24583 5479 28745 28928 18594 3896 37220 30391 40706 2091 810 5134 36772,16,73394,prior
7,230563,43279 7676 27796 31964 3298 31992 17224 38656 594 5322 23909,11,49744,prior
8,2882950,22922 46802 24489 24695 43295 5876 45002,7,127529,prior
9,817584,11182 46667,2,173746,prior


## 3.2. Tokenize sentences

In [46]:
# start pool process for parallel procressing
pool = multiprocessing.Pool(processes = cpus)
cpus

4

In [51]:
# text processing
import nltk 
nltk.download("punkt")
from nltk import sent_tokenize, word_tokenize


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [52]:
# training sentences
t0 = time.time()
train_orders = pool.map(word_tokenize, train["product_id"])
t1 = time.time()
print(len(train_orders))

1000000


In [53]:
# time taken
print(f"Time Taken: {t1 - t0}")

Time Taken: 63.66749143600464


In [54]:
# training orders
train_orders[0:3]

[['36127', '260', '38739', '9839', '31717', '5212', '16797', '26209', '5652'],
 ['34126',
  '34065',
  '405',
  '43772',
  '43129',
  '27966',
  '24964',
  '33731',
  '49520',
  '42736'],
 ['7677',
  '27086',
  '46175',
  '4421',
  '12276',
  '9339',
  '19678',
  '42701',
  '4605',
  '24964']]

In [55]:
# validation sentences
t0 = time.time()
val_orders = pool.map(word_tokenize, val["product_id"])
t1 = time.time()
print(len(val_orders))

38701


----

### In-class exercise

In [111]:
# what do the first three validation orders look like?

In [112]:
# Can you similarly tokenize the test sentences?
#print(len(test_orders))

----

# 4. Word2Vec sample model

## 4.1. Define and train model

In [56]:
#===============================================
# define and train model
#===============================================

w2v_1 = gensim.models.Word2Vec(sentences = train_orders,
                               workers = multiprocessing.cpu_count(),
                               seed = 1234)

## Model properties

In [57]:
#===============================================
# vocabulary
#===============================================

# vocabulary length
f"Vocab length: {len(w2v_1.wv.vocab)}"

'Vocab length: 14340'

In [60]:
# sample vocabulary
list(w2v_1.wv.vocab.keys())[0:5]

['36127', '260', '38739', '9839', '31717']

In [61]:
# length of corpus
print(w2v_1.corpus_count)

1000000


In [62]:
# number of iterations
w2v_1.iter

5

In [118]:
#===============================================
# product vectors
#===============================================

# enter product-id
prod_id = "1"
print(w2v_1[prod_id].shape)
print("------------")
print("------------")
print(w2v_1[prod_id])

(100,)
------------
------------
[ 0.33256823 -0.8674035   0.78945756 -0.8326404  -0.35906547  1.0120238
 -0.06898001  0.2782327  -0.17833115  0.87456757 -0.02812021  1.8168643
  0.30041236  0.3711286  -0.20275462 -0.23503277  1.3704741  -0.5922891
  1.0876763   1.792287   -0.612754   -1.331187    0.64436466 -0.20730683
  1.269803   -1.0397085   0.9416942   1.3022867   0.11127125 -1.0850266
 -0.56653965 -0.06770324 -0.8991397   0.21280771  0.09323756  2.894438
  0.42654473 -1.4698052  -0.5661502   0.37270918  0.13584968  1.2705847
  0.0365853  -0.9683485  -1.5081807  -0.51652515  0.2912822   0.12690805
  0.29624075  0.5983758   1.0610453   0.74076706  0.9236454   0.18861865
 -0.06008022  0.34126574 -0.44573402 -0.26175475  0.20415974  0.77992284
 -0.33681318 -0.11273185  0.07486103 -1.496057   -0.6121371   0.33590174
  0.7643749   0.5597887  -1.2259272  -1.067092    0.7632559   0.09929376
 -0.39279962  0.20848511 -0.3468598   0.23529449  1.0096282  -0.45790723
 -0.37464425 -0.01287276 

## 4.2. Update embeddings

In [63]:
#===============================================
# update model weights
#===============================================
w2v_1.train(sentences = train_orders, total_examples = w2v_1.corpus_count, epochs = 2)

(19307842, 20190222)

## 4.3. Inspect model output

In [64]:
#===============================================
# upadted product vectors
#===============================================

# enter product-id
prod_id = "1"
print(w2v_1[prod_id].shape)
print("------------")
print("------------")
print(w2v_1[prod_id])

(100,)
------------
------------
[-0.15897858  1.3138765  -0.4458094  -1.1327949  -0.52582616 -0.8279934
  0.15774423  0.25105605 -0.4333221  -0.08082795  0.86285216 -0.70886874
  0.7045319  -0.2238739  -0.1918213   0.72482157 -0.8717197  -0.5153289
  0.83717036 -0.31201124 -0.82710207 -0.08100231 -0.69025236  0.7978647
 -1.2918801   0.12263945 -0.4358498   0.64619535  0.26545691  0.18796475
 -1.0406277  -0.95823187  1.5320331  -0.6381243   0.55556875  0.81362915
  0.725703   -0.59123546  1.5706028   1.0049564   0.73284686  0.38947198
  0.5179963  -1.1165116  -0.5719591  -0.5047214   0.19921948 -1.0657848
 -0.6876577  -0.30792212  0.03496341  1.1194674  -0.6617525  -0.35422048
 -0.25476882 -0.01683265 -0.23153007 -0.1554616   1.59474     2.0798566
 -0.91083187 -0.01950055 -0.59591573 -0.82973194 -0.53231645  0.93636
  0.17496532  1.453717    1.6236032   0.0149919   0.9578527  -0.25117806
  0.59807855 -0.7706782   0.5603701   2.2639897  -0.8622636  -0.20782329
 -1.5082802   1.3553026   

--------

# 5. Improve W2V model

In [68]:
#===============================================
# model parameters
#===============================================

# size of embedding matrix
emb_size = 100

# context window size
cxt_window = 10

# batch size for gradient update
batch_size = 10000

# down-sample high frequency words
hfs = 0.001

# learning rate
lr = 0.05

In [69]:
#===============================================
# define model
#===============================================

w2v_1 = gensim.models.Word2Vec(sentences = train_orders,
                               size = emb_size, # number of columns in embedding matrix
                               hs = 1, # hierarchical softmax
                               negative = 0, # negative sampling
                               window = cxt_window, # context window
                               min_count = 1, # minimum frequency count
                               batch_words = batch_size, # batch size for update
                               alpha = lr, # learning rate
                               sample = hfs, # down sample high frequency words 
                               workers = cpus,
                               seed = 1234)

## 5.1. Score on validation and test

In [70]:
#===============================================
# Score model to get log-likelihood
#===============================================

def ScoreW2V(test_sent, model, normalize = True, avg_over_sent = True):
    test_score = model.score(test_sent, total_sentences = len(test_sent), 
                             chunksize = 100,
                             queue_factor = 2,
                             report_delay = 1)
    if normalize:
        test_score = [test_score[x]/len(test_sent[x]) for x in range(len(test_sent))]
    else:
        test_score = list(test_score)
    if avg_over_sent:
        test_score = np.mean(test_score)
    return test_score

In [71]:
w2v_1_val = ScoreW2V(test_sent = val_orders, model = w2v_1, normalize = True, avg_over_sent = True)

In [72]:
w2v_1_val

-7.532278745791191

## 5.2. Similar products

In [73]:
#===============================================
# similar products
#===============================================

# sample product
prod_id = "10"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])

,product_id,product_name,aisle_id,department_id,aisle,department


In [74]:
#===============================================
# similarity from model
#===============================================

w2v_1.wv.most_similar(prod_id)

[('44375', 0.7115499973297119),
 ('6475', 0.6679061055183411),
 ('4138', 0.5879172086715698),
 ('4493', 0.5435665845870972),
 ('34024', 0.4911191463470459),
 ('28004', 0.478809118270874),
 ('32380', 0.4643350839614868),
 ('39921', 0.4568260908126831),
 ('15327', 0.4303375482559204),
 ('6474', 0.4241175651550293)]

In [75]:
#===============================================
# lookup product info
#===============================================
most_similar_prods = [y[0] for y in w2v_1.wv.most_similar(positive = prod_id)]
most_similar_prods = products.loc[products["product_id"].isin(most_similar_prods), :]
display(most_similar_prods)

,product_id,product_name,aisle_id,department_id,aisle,department


In [76]:
#===============================================
# most dissimilar products
#===============================================
most_dissimilar_prods = [y[0] for y in w2v_1.wv.most_similar(negative = [prod_id])]
most_dissimilar_prods = products.loc[products["product_id"].isin(most_dissimilar_prods), :]
display(most_dissimilar_prods)

,product_id,product_name,aisle_id,department_id,aisle,department


----

### In-class exercise

In [77]:
# find the most similar products to 
prod_id = "100"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])

,product_id,product_name,aisle_id,department_id,aisle,department


In [131]:
# which products are most similar to peanut better and strawberry jam sandwich

In [132]:
# what about the most dissimilar products?

----

# 5. Product maps using t-sne 

## 5.1. Extract all product vectors

In [78]:
#===============================================
# product vectors
#===============================================

items = [key for key in w2v_1.wv.vocab.keys()] # all product-ids
word_vec = [list(w2v_1[item]) for item in items] # "word" vector for each product id
word_vec = np.array(word_vec) # convert to array
word_vec_df = pd.DataFrame(word_vec) # convert to data frame
w2v_vec_names = ["wv" + str(x + 1) for x in range(word_vec_df.shape[1])] # column names
word_vec_df.columns = w2v_vec_names # assign column names
word_vec_df["product_id"] = items # include product id in data frame
word_vec_df["product_id"] = word_vec_df["product_id"].astype(int) # convert to type integer for later merge
word_vec_df = word_vec_df[["product_id"] + w2v_vec_names] # re-order columns
print(word_vec_df.shape)

(14340, 101)


In [79]:
display(word_vec_df.head(10))

,product_id,wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv8,wv9,wv10,wv11,wv12,wv13,wv14,wv15,wv16,wv17,wv18,wv19,wv20,wv21,wv22,wv23,wv24,wv25,wv26,wv27,wv28,wv29,wv30,wv31,wv32,wv33,wv34,wv35,wv36,wv37,wv38,wv39,wv40,wv41,wv42,wv43,wv44,wv45,wv46,wv47,wv48,wv49,wv50,wv51,wv52,wv53,wv54,wv55,wv56,wv57,wv58,wv59,wv60,wv61,wv62,wv63,wv64,wv65,wv66,wv67,wv68,wv69,wv70,wv71,wv72,wv73,wv74,wv75,wv76,wv77,wv78,wv79,wv80,wv81,wv82,wv83,wv84,wv85,wv86,wv87,wv88,wv89,wv90,wv91,wv92,wv93,wv94,wv95,wv96,wv97,wv98,wv99,wv100
0,36127,-1.323354,0.612752,-0.398217,0.192804,0.471136,-0.007201,0.234444,-0.119251,-0.109560,0.434752,-0.114712,0.604006,-0.584084,-0.729728,-0.668062,0.166142,-0.731873,-0.589978,0.534785,-0.345381,-1.276464,0.126199,-1.374089,1.094494,1.086177,0.045582,-0.749013,0.529887,-0.170947,0.448088,0.277849,0.226307,-0.537723,-0.606602,0.178671,-0.141900,0.641265,0.336838,-0.728748,0.169845,-0.161269,-0.258954,0.624895,-1.670931,-1.957299,-0.484100,-0.220584,0.740474,-0.045081,0.310734,-0.571723,-0.402898,-0.235510,0.055262,-0.920894,-1.011401,-0.575180,-0.533943,0.269430,0.259720,-0.450105,-0.867342,0.311568,1.371603,-0.230373,1.032849,1.345813,-0.169027,1.641874,0.549985,0.704802,-0.688428,-0.241989,-1.421533,0.102905,-0.460917,0.182365,-0.017828,-0.248468,0.044150,-0.321015,-0.567452,-1.468887,1.221046,-0.237686,0.487675,0.233861,0.031875,0.556341,-0.805869,-1.055952,0.386513,-0.373290,-1.271031,-0.508794,0.389508,0.278921,-0.882582,0.721076,0.309440
1,260,-1.004508,-0.630055,0.464731,-0.591676,-0.101519,-0.439685,-0.097783,0.182098,-0.035338,-0.470347,0.360966,-0.373011,0.011665,-0.576346,-0.447226,-0.281747,-0.492026,-1.521371,0.568475,0.019473,0.580504,0.235527,-0.148362,0.054256,0.813160,0.685138,-0.728926,-0.729078,0.351365,0.543991,1.201986,0.257552,-0.444510,-0.157458,0.909715,0.316515,0.091883,-0.221406,0.584473,-0.137114,0.909496,0.697570,-0.163526,0.695530,-0.214333,0.237807,0.133612,-0.515258,-0.303973,0.452173,-0.476972,-1.096938,-0.075441,-0.537928,0.244431,-0.168813,-0.091763,0.077005,-0.285658,-0.402576,0.643075,-0.354100,-0.208810,0.983676,0.497485,0.433619,0.062218,0.421122,0.220806,0.252234,0.134303,-0.352079,1.236468,0.996296,-0.369578,-0.506816,0.665192,0.667269,0.127533,0.448892,-0.436633,0.080658,0.438812,0.260601,-0.137237,-0.604359,-0.425628,-0.234384,-0.319999,0.088704,0.591365,0.419638,1.360114,-0.381888,-0.196939,-0.270422,0.737956,-0.170715,-0.289386,0.099266
2,38739,0.117583,-0.186220,0.670645,0.761699,-0.368010,-0.634671,0.324184,-0.363019,-0.495294,-0.007327,0.582694,-0.094889,-0.156500,0.732701,-0.303630,-0.683933,-0.330770,-0.002054,0.035253,1.271901,0.704630,-0.170999,-1.281010,-0.255003,-0.023034,0.083850,1.093869,-0.638069,-0.021917,0.981816,0.981029,-1.598578,0.678937,0.183380,-0.220802,-0.116040,-0.756623,0.174813,-0.201381,0.545292,1.924214,-0.225527,-0.845195,0.016864,0.113818,-0.280119,0.306540,-1.224081,-0.017970,-0.399201,0.584661,0.355410,1.078108,0.093555,0.049144,0.566148,-0.211600,0.067576,0.343647,-0.126946,-0.115765,0.234162,1.215726,1.065178,-0.525049,0.044128,-0.600766,-0.792392,-0.249204,0.334837,0.529613,0.099851,0.167005,-0.934140,1.915650,-0.472308,-0.402012,0.923185,0.648542,0.800638,-0.844960,-0.268484,-0.532000,-0.825948,-0.122777,-0.139188,0.598380,-0.699188,-0.439457,0.148125,-0.084110,0.155991,-0.529889,0.235420,0.361776,-0.426244,-0.072735,-0.073560,-0.155613,-0.262718
3,9839,0.317504,0.583372,0.504759,-0.539479,0.068734,-0.038911,0.660003,0.769504,0.683357,0.445805,-0.783957,-0.420226,0.384448,0.235110,-0.094279,-0.781080,-1.061411,-1.024354,-0.333595,0.694940,-0.291977,-0.498779,0.159861,-0.284094,-0.215898,0.047123,-0.692410,-0.034591,0.402449,0.812948,1.028750,-0.089558,-0.363315,0.235936,0.259110,-0.240004,-0.087260,-0.075477,-0.364875,-1.188952,-0.703688,-0.501646,-0.345118,-0.289469,-1.236472,1.303083,0.638172,0.158570,0.495495,1.440690,0.656163,0.091888,0.230711,-0.248618,-0.575258,-0.364714,-0.030669,-0.615998,0.079768,-1.014509,-0.384285,-0.218831,0.494353,0.093158

## Merge product vectors with product info

In [80]:
#===============================================
# merge relevant columns
#===============================================

# relevant columns from product info
prod_info_cols = ["product_id", "product_name", "department", "aisle"]

# merge
word_vec_df = pd.merge(products[prod_info_cols], word_vec_df, on = "product_id", how = "inner")
print(word_vec_df.shape)

(14340, 104)


In [136]:
display(word_vec_df.head(10))

,product_id,product_name,department,aisle,wv1,wv2,wv3,wv4,wv5,wv6,wv7,wv8,wv9,wv10,wv11,wv12,wv13,wv14,wv15,wv16,wv17,wv18,wv19,wv20,wv21,wv22,wv23,wv24,wv25,wv26,wv27,wv28,wv29,wv30,wv31,wv32,wv33,wv34,wv35,wv36,wv37,wv38,wv39,wv40,wv41,wv42,wv43,wv44,wv45,wv46,wv47,wv48,wv49,wv50,wv51,wv52,wv53,wv54,wv55,wv56,wv57,wv58,wv59,wv60,wv61,wv62,wv63,wv64,wv65,wv66,wv67,wv68,wv69,wv70,wv71,wv72,wv73,wv74,wv75,wv76,wv77,wv78,wv79,wv80,wv81,wv82,wv83,wv84,wv85,wv86,wv87,wv88,wv89,wv90,wv91,wv92,wv93,wv94,wv95,wv96,wv97,wv98,wv99,wv100
0,1,Chocolate Sandwich Cookies,snacks,cookies cakes,0.744708,1.295512,-1.193265,0.598794,-0.641248,1.223336,0.919235,0.809076,0.790224,-1.406452,0.579844,-0.417504,0.320172,-1.006021,-1.247783,0.728549,0.282868,-0.163652,0.442921,0.128002,1.018741,-1.040465,0.126721,0.412622,0.333163,0.891142,-0.438378,-0.414526,-0.543069,-1.434892,-0.369650,-0.234173,-0.450169,0.144263,-0.889828,-0.092579,0.095353,-0.893399,-0.008923,-0.703940,-0.921108,-1.429784,0.343726,-0.504728,-1.583240,0.781951,-0.361618,-0.487692,0.108233,-1.003854,0.445510,0.412661,2.036789,0.664207,-0.936389,0.153528,-0.561427,0.708672,-1.219241,-0.778229,0.468867,-0.237254,-0.688795,0.463018,0.221844,-1.234477,-0.520427,0.230692,-1.529698,0.867693,-0.802470,-0.222265,-0.507642,0.082833,-0.856651,0.124394,-0.133245,-1.038114,-0.809275,-0.205799,-0.915241,-0.091222,-0.302496,0.530782,0.226141,-0.134771,-0.599206,-0.370918,0.447913,0.089710,-0.735691,-0.266758,-1.274789,-1.471167,0.414221,-0.482969,-0.722963,-0.789088,-0.890457,0.141746
1,3,Robust Golden Unsweetened Oolong Tea,beverages,tea,-0.156999,-0.097948,1.033049,-0.075712,-0.156379,0.859535,-0.202249,-0.692840,1.155985,-0.154656,1.157791,-0.287568,0.291294,0.716055,-0.463947,-0.266470,-0.432250,-0.462481,-0.609656,0.939700,0.484884,-0.209098,0.338069,1.126593,-0.584675,-1.762120,-1.089806,1.540631,0.531554,0.231101,0.813328,0.308757,-1.116319,0.098051,0.718829,0.123319,-0.361023,-0.134135,0.584352,-0.445583,0.308385,0.375197,-0.108583,-1.362428,0.132718,-1.448387,-0.245276,1.475555,1.410261,-0.275512,0.245381,-0.178379,0.160996,-0.317731,-0.286790,-0.621966,1.150026,-1.059867,-1.085093,-0.115807,0.323545,-1.090988,0.613781,0.329271,1.433473,0.560549,-0.153129,-0.580917,-1.053341,0.052376,-0.343750,1.419617,0.731421,0.625508,0.882422,-0.762834,-0.411799,0.003922,-1.166005,-0.027614,0.316082,-0.358464,-0.382854,1.930140,-0.251959,0.190719,0.468010,0.382750,0.392147,-0.921297,0.853766,0.890511,0.992793,0.780575,0.511955,0.541127,-0.399190,0.485569,-0.989308,-0.412728
2,4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,frozen,frozen meals,0.632169,-0.042714,0.447641,0.565454,-0.173308,-1.259952,-0.374098,0.565182,-0.501129,-0.326884,-1.161234,0.588260,1.618659,0.294504,0.154790,0.866748,-1.177649,-0.134442,-0.567716,-0.921026,0.309276,-0.598171,2.971751,-0.787572,-0.649750,-0.231039,1.806901,1.559718,0.501790,0.281565,0.014173,0.633412,0.325695,0.359038,-0.070100,0.124627,-0.962404,-0.134815,-0.145717,1.183958,0.241399,0.964502,1.174005,1.029604,0.155260,-0.624118,-0.044555,0.265823,1.074033,-1.058899,-0.206956,0.068395,1.224924,0.802075,-0.673816,2.257773,-1.068509,0.063531,1.522198,0.739853,1.355033,0.423484,0.501739,0.359911,0.319578,-0.454862,0.050564,0.328704,0.205272,-0.856989,0.670041,1.876261,1.597733,1.334862,-0.317515,-0.957292,-1.040504,-0.442202,-0.018599,0.246100,0.799682,-0.482413,-1.251701,-1.626282,-0.359774,0.296589,-0.049335,0.718455,1.660907,-0.985967,2.556258,-0.344786,-0.415751,-1.505674,-0.344438,0.000373,-0.931319,2.007142,-0.076104,-0.533512
3,10,Sparkling Orange Juice & Prickly Pear Beverage,beverages,water seltzer sparkling water,-0.253488,0.374834,-1.069590,0.956164,-0.311371,1.789404,0.267260,0.387516,-0.654127,1.027248,1.398794,0.410207,0.221405,1.188907,0.842219,0.270739,-1.056333,-1.884760,1.011292,0.223562,-0.034032,-0.055429,0.418091,0.480409,1.238376,0.885619,0.360691,3.544943,-0.051665,-0.017406,-0.987518,0.134707,-0.523529,0.258269,-0.3047

## 5.2. Fit t-sne

In [81]:
# =================================================
# fit t-sne
# =================================================
tsne = TSNE(n_components = 2, verbose = 1, perplexity = 35, n_iter = 400)

In [ ]:
# fit
t0 = time.time()
tsne_fit = tsne.fit_transform(word_vec_df[w2v_vec_names])
t1 = time.time()

[t-SNE] Computing 106 nearest neighbors...
[t-SNE] Indexed 14340 samples in 0.242s...


In [139]:
# time taken
f"Time Taken: {t1 - t0}"

'Time Taken: 187.86533784866333'

## T-sne component data frame

In [140]:
#===============================================
# create t-sne data frame
#===============================================

tsne_df = word_vec_df[["product_name", "department", "aisle"]]

# extract t-sne dimensions
tsne_df["x_tsne"] = tsne_fit[:,0]
tsne_df["y_tsne"] = tsne_fit[:,1]
print(tsne_df.describe())

             x_tsne        y_tsne
count  14340.000000  14340.000000
mean   0.081110     -0.013953    
std    10.179037     7.783475    
min   -22.075830    -19.978592   
25%   -7.855527     -5.532616    
50%   -0.820201     -0.028690    
75%    8.695611      5.579124    
max    22.841021     21.091007   


## Plot with ggplot

In [141]:
#===============================================
# subset data for plot
#===============================================

# select only top departments
select_dept = ["produce", "babies", "beverages"]
tsne_plot_df = tsne_df.loc[tsne_df["department"].isin(select_dept), :]
print(tsne_plot_df.shape)

(2726, 5)


In [ ]:
tsne_plot = ggplot(tsne_plot_df, aes(x = "x_tsne", y = "y_tsne", color = "department") ) \
        + geom_point(size = 70, alpha = 0.5) \
        + ggtitle("T-sne on product vectors") \
        + xlab(" ") + ylab(" ")
tsne_plot

---

## In-class exercise

## Learning embeddings using Skip-gram

- We will now use another method to train embeddings called skip-gram

In [ ]:
#===============================================
# model parameters
#===============================================

# size of embedding matrix
emb_size  # set between 50-100

# context window size
cxt_window # set between 2-10 

# batch size for gradient update
batch_size # set between 2000 to 10000

# learning rate
lr # set between 0.001 to 0.01

In [ ]:
#===============================================
# define and train model
#===============================================

t0 = time.time()
### Write model code here
t1 = time.time()

In [ ]:
# time taken
f"Time Taken: {t1 - t0}"

In [ ]:
# find the most similar products to 
prod_id = "1000"

# product info for sample product
display(products.loc[products["product_id"].isin([prod_id]), :])